In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from ib_insync import util, IB, LimitOrder
util.startLoop()

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>
from src.utils import Vars, get_pickle, place_orders

_vars = Vars(MARKET)
PORT = port = _vars.PORT

DATAPATH = ROOT / 'data' / MARKET

In [ ]:
import asyncio

from src.nse import Vars, create_target_opts
from src.utils import (Vars, get_pickle, 
                       chunk_me, flatten,
                       get_prec, pickle_with_age_check,)

from tqdm.asyncio import tqdm
import pandas as pd
import random

In [ ]:
from utils import place_orders

In [ ]:
df_tgt = get_pickle(DATAPATH / "df_naked_targets.pkl")
df = df_tgt[_vars.OPT_COLS[0]].dropna(subset=['expPrice'])
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]

In [ ]:
with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    trades = ib.trades()

In [ ]:
trades

In [ ]:
with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    # ib.sleep(0.3)
    trades = ib.trades()

if trades:
    all_trades_df = (util.df(t.contract for t in trades).join(
        util.df(t.orderStatus
                for t in trades)).join(util.df(t.order for t in trades),
                                        lsuffix="_"))

    all_trades_df.rename({"lastTradeDateOrContractMonth": "expiry"},
                            axis="columns",
                            inplace=True)
    trades_cols = [
        "conId",
        "symbol",
        "secType",
        "expiry",
        "strike",
        "right",
        "orderId",
        "permId",
        "action",
        "totalQuantity",
        "lmtPrice",
        "status",
    ]

    dfo = all_trades_df[trades_cols]
    df_openords = dfo[all_trades_df.status.isin(_vars. ACTIVE_STATUS)]